In [1]:
import evalFunctions as ef
import os
import json
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from pyvis.network import Network
import networkx as nx

IndentationError: unexpected indent (evalFunctions.py, line 83)

In [ ]:
stop_words = set(stopwords.words('english'))

pathlist = []
for i in range(0,7):
    filename = 'sentenceswithnames'+str(i)+'.txt'
    filepath = os.path.join('Data','RESULTS',filename)
    if os.path.exists(filepath) and not os.path.exists(os.path.join('Data','RESULTS','unigram_mtx_'+str(i)+'.txt')): 
        pathlist.append(filepath)        
        # Statistischee Analysen gesamter Text
        with open(filepath,'r',encoding='utf-8') as f:
            content = f.read()
        # Sätze erstellen
        sents = nltk.sent_tokenize(content) # Output is a list of sentences
        print(len(sents))
        # print(type(sents))
        # print(sents)
        unigrammtx = ef.unigramMatrix(sents)
        print(len(unigrammtx))
        with open(os.path.join('Data','RESULTS','unigram_mtx_'+str(i)+'.txt'),'w',encoding='utf-8') as f:
            for word in unigrammtx:
                f.write(word)
                f.write('\n')

# os.remove(os.path.join('Data','RESULTS','unigram_mtx.txt'))
# print(pathlist)
# with open('unigram_mtx.txt','a') as f:
#     for file in pathlist:
#         with open(file) as input:
#             f.write(input.read())

In [ ]:
with open(os.path.join('Data','RESULTS','unigram_mtx.txt'),'w',encoding='utf-8') as f:
        f.write(str(file)+'\n')

ngram_mtx = []
with open(os.path.join('Data','RESULTS','unigram_mtx.txt'),'r',encoding='utf-8') as f:
    content = f.readlines()
for line in content:
    ngram_mtx.append(line)
# Filter auf die Unigramme
uallmtx = ef.ngramFilter(ngram_mtx,1,0,0)


In [ ]:

# Ngramme initialisieren
# print('unimtx')
# unimtx_freq,mtx_bigram = ef.initNgrams(unigrammtx)
# print('uallmtx')
# uall_fmtxreq,uallmtx_bigram = ef.initNgrams(uallmtx)
# print('unamesmtx')
# unamesmtx_freq,unamesmtx_bigram = ef.initNgrams(unamesmtx)
# print('uadmtx')
# uadmtx_freq,uadmtx_bigram = ef.initNgrams(uadmtx)

In [ ]:
def createNetworkGraph(bigram):
    # Gewichtung
    weight = {}
    # Tuple-Dictionaries
    connection = {}
    connection_f = {}
    connection_m = {}
    # Filter values
    fv_names,fv_adv,fv_adj,female,male = ef.getFilterValues(1,0,0,1,1) # funktionuggelt
    # Graphen
    g = nx.Graph()
    g_f = nx.Graph()
    g_m = nx.Graph()
    # Kennzahlen
    weight_of_nodes = []
    # Farben Variablen
    lblue = '#85B4E4'
    dgreen = '#0F8436'
    for tuple in bigram:
        weight = 0
        # TODO Gewichtung überprüfen
        # TODO Graphen umstrukturieren
        if tuple[0] in fv_names and not tuple[1] in fv_names:
            # weiblich
            if tuple[0] in female:
                connection_f.update({tuple:weight})
                weight = connection_f.get(tuple)
                weight += 1
                connection_f.update({tuple:weight})
            # männlich
            if tuple[0] in male:
                connection_m.update({tuple:weight})
                weight = connection_m.get(tuple)
                weight += 1
                connection_m.update({tuple:weight})
            # allgemein
            connection.update({tuple:weight})
            weight = connection.get(tuple)
            weight += 1
            connection.update({tuple:weight})
        if tuple[1] in fv_names and not tuple[0] in fv_names:
            # weiblich
            if tuple[0] in female:
                n_tuple = (tuple[1],tuple[0])
                connection_f.update({n_tuple:weight})
                weight = connection_f.get(n_tuple)
                weight += 1
                connection_f.update({n_tuple:weight})
            # männlich
            if tuple[0] in male:
                n_tuple = (tuple[1],tuple[0])
                connection_m.update({n_tuple:weight})
                weight = connection_m.get(n_tuple)
                weight += 1
                connection_m.update({n_tuple:weight})
            # allgemein
            n_tuple = (tuple[1],tuple[0])
            connection.update({n_tuple:weight})
            weight = connection.get(n_tuple)
            weight += 1
            connection.update({n_tuple:weight})
    # Alle Personen-Graph
    color_map = []
    l_of_names = []
    l_of_words = []
    for key in connection:
        value = connection.get(key)
        g.add_node(key[0], cluster='noun')
        g.add_node(key[1], cluster='ad')
        g.add_edge(key[1],key[0],weight=value)
        weight_of_nodes.append(value)
        # Color-Mapping
        if key[1] in fv_names: # Paar aus Namen
            # Name 1 bereits erfasst?
            if not key[0] in l_of_names: # Wenn nein
                color_map.append(dgreen) # TODO Farbe ändern
                l_of_names.append(key[0])
            # Name 2 bereits erfasst?
            if not key[1] in l_of_names: # Wenn nein
                color_map.append(dgreen)
                l_of_names.append(key[1])
        else: # nur 1 Name
            if not key[0] in l_of_names:
                color_map.append(dgreen)
                l_of_names.append(key[0])
            #Color-Mapping für alle anderen Wörter
            if not key[1] in l_of_words: 
                color_map.append(lblue)
                l_of_words.append(key[1])
    # Assortivität-Koeffizient
    cr = nx.attribute_assortativity_coefficient(g,"cluster",nodes=key)
    # Reciprocity
    rec = nx.overall_reciprocity(g)
    # Transitivity
    trans = nx.transitivity(g)
    # Clustering
    clu = nx.clustering(g)
    avgcluster = nx.average_clustering(g)
    gdeg = nx.generalized_degree(g)
    
    print(cr, rec, clu, avgcluster, gdeg)
    # Kennzahlen und Algos
    # print(weight_of_nodes)
    # Graphen darstellen
    fig = plt.figure(figsize=(50,20))
    subax1 = plt.subplot(122)
    nx.draw(g, node_color=color_map, edge_color ='grey', with_labels=True)
    subax1.set_title(f"Wortverknüpfungen aller Personen\nAssortivitätskoeffizient: {cr}\nReciprocity: {rec}")
    # plt.savefig('wordconnectivity_women.png')
    # Frauen-Graph
    color_map = []
    l_of_names = []
    l_of_words = []
    weight_of_nodes = []
    for key in connection_f:
        value = connection_f.get(key)
        g_f.add_node(key[0], cluster='noun')
        g_f.add_node(key[1], cluster='ad')
        g_f.add_edge(key[0],key[1],weight=value)
        weight_of_nodes.append(value)
        # Color-Mapping
        if key[1] in fv_names: # Paar aus Namen
            # Name 1 bereits erfasst?
            if not key[0] in l_of_names: # Wenn nein
                color_map.append(dgreen) # TODO Farbe ändern
                l_of_names.append(key[0])
            # Name 2 bereits erfasst?
            if not key[1] in l_of_names: # Wenn nein
                color_map.append(dgreen)
                l_of_names.append(key[1])
        else: # nur 1 Name
            if not key[0] in l_of_names:
                color_map.append(dgreen)
                l_of_names.append(key[0])
            #Color-Mapping für alle anderen Wörter
            if not key[1] in l_of_words: 
                color_map.append(lblue)
                l_of_words.append(key[1])
    # assortativity_coefficient
    cr_f = nx.attribute_assortativity_coefficient(g_f,"cluster",nodes=key)
    rec_f = nx.overall_reciprocity(g_f)
    print(cr_f, rec_f)
    # Kennzahlen & Algos
    # print(weight_of_nodes)
    fig = plt.figure(figsize= (50,20))
    subax1 = plt.subplot(122)
    nx.draw(g_f, node_color=color_map, edge_color ='grey', with_labels=True)
    subax1.set_title(f"Wortverknüpfungen der weiblichen Personen\nAssortivitätskoeffizient: {cr_f}\nReciprocity: {rec_f}")
    # plt.savefig('wordconnectivity_women.png')
    # Männer-Graph
    color_map = []
    l_of_names = []
    l_of_words = []
    weight_of_nodes = []
    for key in connection_m:
        value = connection_m.get(key)
        g_m.add_node(key[0], cluster='noun', color='red')
        g_m.add_node(key[1], cluster='ad', color='red')
        g_m.add_edge(key[0],key[1],weight=value)
        weight_of_nodes.append(value)
    # Color-Mapping
        if key[1] in fv_names: # Paar aus Namen
            # Name 1 bereits erfasst?
            if not key[0] in l_of_names: # Wenn nein
                color_map.append(dgreen)
                l_of_names.append(key[0])
            # Name 2 bereits erfasst?
            if not key[1] in l_of_names: # Wenn nein
                color_map.append(dgreen)
                l_of_names.append(key[1])
        else: # nur 1 Name
            if not key[0] in l_of_names:
                color_map.append(dgreen)
                l_of_names.append(key[0])
            #Color-Mapping für alle anderen Wörter
            if not key[1] in l_of_words: 
                color_map.append(lblue)
                l_of_words.append(key[1])
    # assortativity_coefficient
    cr_m = nx.attribute_assortativity_coefficient(g_m,"cluster",nodes=key)
    rec_m = nx.overall_reciprocity(g_m)
    print(cr_m, rec_m)
    # Kennzahlen & Algos
    # print(weight_of_nodes)
    fig = plt.figure(figsize= (50,20))
    subax1 = plt.subplot(122)
    nx.draw(g_m, node_color=color_map, edge_color ='grey', with_labels=True)
    subax1.set_title(f"Wortverknüpfungen der männlichen Personen\nAssortivitätskoeffizient: {cr_m}\nReciprocity: {rec_m}")
    # plt.savefig('wordconnectivity_men.png')

# wordnet = nltk.pywordnet

createNetworkGraph(uallmtx_bigram)

NameError: name 'uallmtx_bigram' is not defined